In [1]:
from IPython.core.display import display, HTML

import pandas as pd
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import gc
import time

from joblib import Parallel, delayed

from sklearn import preprocessing, model_selection
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt 
import seaborn as sns
import numpy.matlib


path_submissions = '/'

target_name = 'target'
scores_folds = {}

In [2]:
import pickle
train1 = pd.read_pickle("temp/train3.pkl")
test1 = pd.read_pickle("temp/test3.pkl")
train_nn = pd.read_pickle("temp/train_nn.pkl")
test_nn = pd.read_pickle("temp/test_nn.pkl")

# values = []
# file=open(r"temp/values.pkl","rb")
# values = pickle.load(file)
# file.close()

In [3]:
import torch
from torch import nn
import torch.nn.functional as F

class Act_op(nn.Module):
    def __init__(self):
        super(Act_op, self).__init__()
 
    def forward(self, x):
        x = x * torch.sigmoid(x)
        return x

In [4]:
# import EarlyStopping
from pytorchtools import EarlyStopping
early_stopping = EarlyStopping(20, verbose=False)

In [5]:
#利用pytorch构建NN
hidden_units = (128,64,32)
stock_embedding_size = 24

cat_data = train_nn['stock_id']
n_cata = max(cat_data)

class NNet(nn.Module):
    def __init__(self,n_features,n_cata,stock_em):
        super(NNet, self).__init__()
        self.flatten = nn.Flatten()
        self.n_features = n_features
        self.n_cata = n_cata
        self.stock_em = stock_em
        self.Act = Act_op()
        self.f1 = torch.nn.Embedding(self.n_cata+1, self.stock_em)
        self.linear_stack = nn.Sequential(
            nn.Linear(24+n_features, 128),
            self.Act,
            nn.Linear(128, 64),
            self.Act,
            nn.Linear(64, 32),
            self.Act,
            nn.Linear(32,1)
        )
#         self.linear_stack = nn.Linear(24+n_features, 1)

    def forward(self, x_cat, x_num):
        stock_embedded = self.f1(x_cat)
        stock_flattened = self.flatten(stock_embedded)
        out = torch.cat((stock_flattened, x_num),1).float()
        out = self.linear_stack(out)
        return out

In [6]:
class myDataset(torch.utils.data.Dataset):
    def __init__(self, x_cat: np.ndarray, x_num: np.ndarray, y:np.ndarray):
        super().__init__()
        self.x_cat = x_cat
        self.x_num = x_num
        self.y = y

    def __len__(self):
        return len(self.x_num)

    def __getitem__(self, idx):
        return [list(self.x_cat[idx:idx+1]),self.x_num[idx:idx+1][0,:]], list(self.y[idx:idx+1])

In [7]:
class My_loss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, y_true, y_pred):
        return torch.sqrt(torch.mean(torch.pow(((y_true-y_pred)/y_pred),2)))

In [8]:
# Function to calculate the root mean squared percentage error
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

# Function to early stop with root mean squared percentage error
def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

In [9]:
target_name='target'
scores_folds = {}
model_name = 'NN'
pred_name = 'pred_{}'.format(model_name)

n_folds = 5
kf = model_selection.KFold(n_splits=n_folds, shuffle=True, random_state=2021)
scores_folds[model_name] = []
counter = 1

features_to_consider = list(train_nn)

features_to_consider.remove('time_id')
features_to_consider.remove('target')
try:
    features_to_consider.remove('pred_NN')
except:
    pass


train_nn[features_to_consider] = train_nn[features_to_consider].fillna(train_nn[features_to_consider].mean())
test_nn[features_to_consider] = test_nn[features_to_consider].fillna(train_nn[features_to_consider].mean())

train_nn[pred_name] = 0
test_nn[target_name] = 0
test_predictions_nn = np.zeros(test_nn.shape[0])

MemoryError: Unable to allocate 242. MiB for an array with shape (74, 428932) and data type float64

In [ ]:
n_folds = 5
n_count = 1
indexes = np.arange(n_folds).astype(int)
indexes = np.delete(indexes,obj=n_count, axis=0)
indexes = np.r_[values[indexes[0]],values[indexes[1]],values[indexes[2]],values[indexes[3]]]

In [ ]:
indexes

In [ ]:
for n_count in range(n_folds):
    print('CV {}/{}'.format(counter, n_folds))
    
    indexes = np.arange(n_folds).astype(int)    
    indexes = np.delete(indexes,obj=n_count, axis=0) 
    
    indexes = np.r_[values[indexes[0]],values[indexes[1]],values[indexes[2]],values[indexes[3]]]
    
    X_train = train_nn.loc[train_nn.time_id.isin(indexes), features_to_consider]
    y_train = train_nn.loc[train_nn.time_id.isin(indexes), target_name]
    X_test = train_nn.loc[train_nn.time_id.isin(values[n_count]), features_to_consider]
    y_test = train_nn.loc[train_nn.time_id.isin(values[n_count]), target_name]
    
    #############################################################################################
    # NN
    #############################################################################################
    
    try:
        features_to_consider.remove('stock_id')
    except:
        pass

    num_data = X_train[features_to_consider]

    scaler = MinMaxScaler(feature_range=(-1, 1))         
    num_data = scaler.fit_transform(num_data.values)    

    cat_data = X_train['stock_id']    
    target =  y_train

    num_data_test = X_test[features_to_consider]
    num_data_test = scaler.transform(num_data_test.values)
    cat_data_test = X_test['stock_id']

    
    train_dataset = myDataset(cat_data,num_data, y_train)
    test_dataset =  myDataset(cat_data_test, num_data_test, y_test)
    
    os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
    n_epochs = 100
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 512,
                                           num_workers = 0)
    test_loader  = torch.utils.data.DataLoader(test_dataset , batch_size = 512,
                                           num_workers = 0)


    stock_shape = len(cat_data)
    num_input   = len(num_data)
    n_f = len(features_to_consider)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = NNet(n_f,n_cata, stock_embedding_size)
    model.to(device)
    criterion = My_loss().to(device)
    optimizer = torch.optim.Adam(params = model.parameters(), lr = 0.006)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

    for epoch in range (n_epochs):
        train_loss = 0.0

        for data,target in train_loader:
            optimizer.zero_grad()
            x_cat, x_num = data
            x_cat = x_cat[0].cuda()
            x_num = x_num.cuda()
            target = target[0].cuda()
            output = model(x_cat,x_num)
            loss = criterion(output[:,0], target)
            loss.backward()

            optimizer.step()
            train_loss += loss.item()*x_cat.size(0)
        train_loss = train_loss / len(train_loader.dataset)
        print('Epoch:  {}  \tTraining Loss: {:.6f}'.format(epoch + 1, train_loss))
        preds = model(torch.LongTensor(list(cat_data_test)).cuda(), torch.FloatTensor(num_data_test).cuda()
                 ).reshape(1,-1)[0]
        val_loss = criterion(preds, torch.Tensor(y_test.values).cuda())
        scheduler.step(val_loss)
        early_stopping(val_loss, model)
        if early_stopping.early_stop:
            print("Early stopping")
            break

    preds = model(torch.LongTensor(list(cat_data_test)).cuda(), torch.FloatTensor(num_data_test).cuda()
                 ).reshape(1,-1)[0]
    
    score = round(rmspe(y_true = y_test.values, y_pred = preds.cpu().detach().numpy()),5)
    print('Fold {} {}: {}'.format(counter, model_name, score))
    scores_folds[model_name].append(score)

    tt =scaler.transform(test_nn[features_to_consider].values)
    #test_nn[target_name] += model.predict([test_nn['stock_id'], tt]).reshape(1,-1)[0].clip(0,1e10)

    pred_1 = model(torch.LongTensor(list(test_nn['stock_id'])).cuda(), torch.FloatTensor(tt).cuda()
                 ).reshape(1,-1)[0]

    test_predictions_nn += pred_1.cpu().detach().numpy()/n_folds
    #test[target_name] += model.predict([test['stock_id'], test[features_to_consider]]).reshape(1,-1)[0].clip(0,1e10)

    counter += 1
    features_to_consider.append('stock_id')

In [14]:
test_predictions_nn

array([0.0144529 , 0.00183588, 0.00183588])

In [ ]:
t_p = pd.read_csv('input/optiver-realized-volatility-prediction/train.csv')

In [15]:
torch.cuda.empty_cache()

In [2]:
!nvidia-smi

Wed Oct 13 01:33:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.80       Driver Version: 460.80       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:01:00.0 Off |                  N/A |
| 23%   33C    P8    17W / 250W |   3159MiB / 11177MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [19]:
import torch
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [ ]:
from sklearn.model_selection import KFold
scores_folds = []
n_folds = 2
kfold = KFold(n_splits = 2, random_state = 2021, shuffle = True)
for fold, (trn_ind, val_ind) in enumerate(kfold.split(train_nn)):
    print('CV {}/{}'.format(fold+1, n_folds))
    
    
    X_train = train_nn.iloc[trn_ind]
    y_train = train_nn.iloc[trn_ind]
    X_test = train_nn.iloc[val_ind]
    y_test = train_nn.iloc[val_ind]
    
    X_train = train_nn.loc[:,features_to_consider]
    y_train = train_nn.loc[:,target_name]
    X_test = train_nn.loc[:,features_to_consider]
    y_test = train_nn.loc[:,target_name]
    
    #############################################################################################
    # NN
    #############################################################################################
    
    try:
        features_to_consider.remove('stock_id')
    except:
        pass

    num_data = X_train[features_to_consider]

    scaler = MinMaxScaler(feature_range=(-1, 1))         
    num_data = scaler.fit_transform(num_data.values)    

    cat_data = X_train['stock_id']    
    target =  y_train

    num_data_test = X_test[features_to_consider]
    num_data_test = scaler.transform(num_data_test.values)
    cat_data_test = X_test['stock_id']

    
    train_dataset = myDataset(cat_data,num_data, y_train)
    test_dataset =  myDataset(cat_data_test, num_data_test, y_test)
    
    os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
    n_epochs = 5
    
    loadtime = time.time()
    
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 1024 * 1,
                                           num_workers = 0)
    test_loader  = torch.utils.data.DataLoader(test_dataset , batch_size = 1024 * 1,
                                           num_workers = 0)
    
    print("Loading using "+str(time.time()-loadtime)+'s')

    stock_shape = len(cat_data)
    num_input   = len(num_data)
    n_f = len(features_to_consider)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = NNet(n_f,n_cata, stock_embedding_size)
    model.to(device)
    criterion = My_loss().to(device)
    optimizer = torch.optim.Adam(params = model.parameters(), lr = 0.006)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

    for epoch in range (n_epochs):
        train_loss = 0.0
        epoch_time = time.time()
        dtime = []
        Ptime = []
        counter = 0

        for data,target in train_loader:
            optimizer.zero_grad()
            
            datatime=time.time()
            
            x_cat, x_num = data
            x_cat = x_cat[0].cuda()
            x_num = x_num.cuda()
            target = target[0].cuda()
            
            dtime.append(time.time()-datatime)
            predtime=time.time()
            
            output = model(x_cat,x_num)
            loss = criterion(output[:,0], target)
            loss.backward()
            Ptime.append(time.time()-predtime)

            optimizer.step()
            train_loss += loss.item()*x_cat.size(0)
            del x_cat
            torch.cuda.empty_cache()
            counter += 1
        train_loss = train_loss / len(train_loader.dataset)
        print('Epoch:  {}  \tTraining Loss: {:.6f}'.format(epoch + 1, train_loss))
        preds = model(torch.LongTensor(list(cat_data_test)).cuda(), torch.FloatTensor(num_data_test).cuda()
                 ).reshape(1,-1)[0]
        val_loss = criterion(preds, torch.Tensor(y_test.values).cuda())
        scheduler.step(val_loss)
        early_stopping(val_loss, model)
        if early_stopping.early_stop:
            print("Early stopping")
            break
        t1 = time.time()
        del val_loss, preds, train_loss, data,target
        dtime = np.array(dtime).mean()
        Ptime = np.array(Ptime).mean()
        print('This Epoch spend: '+str(time.time()-epoch_time)+'s')
        print('dtime'+str(dtime)+'s',' Proportion:',dtime*counter/(time.time()-epoch_time))
        print('Predtime'+str(Ptime)+'s',' Proportion:',Ptime*counter/(time.time()-epoch_time))
        print('Del using'+str(t1-time.time())+'s')
        
    preds = model(torch.LongTensor(list(cat_data_test)).cuda(), torch.FloatTensor(num_data_test).cuda()
                 ).reshape(1,-1)[0]
    
    score = round(rmspe(y_true = y_test.values, y_pred = preds.cpu().detach().numpy()),5)
    print('Fold {} {}: {}'.format(fold, model_name, score))
    scores_folds.append(score)
    features_to_consider.append('stock_id')

CV 1/2
Loading using 0.0010006427764892578s
Epoch:  1  	Training Loss: nan
This Epoch spend: 36.172807455062866s
dtime0.0006210633280169139s  Proportion: 0.007193954595931284
Predtime0.011768329684091354s  Proportion: 0.13631593687495572
Del using0.0s
Epoch:  2  	Training Loss: nan
This Epoch spend: 35.90600919723511s
dtime0.0005859765346408744s  Proportion: 0.0068379687273469715
Predtime0.008587671065956426s  Proportion: 0.10020979926604254
Del using-0.0009999275207519531s
Epoch:  3  	Training Loss: nan
This Epoch spend: 40.63803505897522s
dtime0.0006162386236441163s  Proportion: 0.006353614089187027
Predtime0.009256149533255858s  Proportion: 0.09543381383390807
Del using-0.0010004043579101562s
Epoch:  4  	Training Loss: nan
This Epoch spend: 76.26059699058533s
dtime0.0005301569981905043s  Proportion: 0.002912851341423997
Predtime0.029511960992380657s  Proportion: 0.1621481098205154
Del using0.0s
Epoch:  5  	Training Loss: nan
This Epoch spend: 43.863324880599976s
dtime0.0005806328857